# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [60]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [61]:
data = pd.read_csv('../data/customer_product_sales.csv')
data

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
...,...,...,...,...,...,...,...
68579,94547,Gerald,Donovan,687884,211,Wine - Prosecco Valdobiaddene,24
68580,57654,Lashonda,Roy,4373415,211,Wine - Prosecco Valdobiaddene,15
68581,36751,Glen,Frederick,2405597,211,Wine - Prosecco Valdobiaddene,10
68582,43270,Devon,Mc Daniel,4328852,211,Wine - Prosecco Valdobiaddene,11


In [62]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [63]:
total_quantity = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'})
total_quantity.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [67]:
product_by_customer_matrix=pd.pivot_table(data, index='ProductName', values='Quantity', columns='CustomerID').fillna(0)
product_by_customer_matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [68]:
customer_similarity_matrix=pd.DataFrame(1/(1 + squareform(pdist(product_by_customer_matrix.T, 'euclidean'))), 
                         index=product_by_customer_matrix.columns, columns=product_by_customer_matrix.columns)
customer_similarity_matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [78]:
# choose randomly Customer with ID 200

top_5_similar_customers=customer_similarity_matrix[200].sort_values(ascending=False)[1:6]
top_5_similar_customers

CustomerID
1920    0.094414
1072    0.092610
3317    0.090094
3253    0.090094
3909    0.089695
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [79]:
records = total_quantity.loc[top_5_similar_customers.index]
records

Quantity
CustomerID ProductName                             
1920       Apricots - Halves                      1
           Bacardi Breezer - Tropical             1
           Barramundi                             1
           Beans - Kidney White                   1
           Beef - Texas Style Burger              1
...                                             ...
3909       Wine - Charddonnay Errazuriz           2
           Wine - Fume Blanc Fetzer               1
           Wine - Magnotta, Merlot Sr Vqa         1
           Wine - Toasted Head                    1
           Wine - Valpolicella Masi               1

[261 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [83]:
aggregate = records.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
aggregate

,Quantity
ProductName,
Soup - Campbells Bean Medley,3
Lamb - Ground,3
Pomello,3
Bay Leaf,3
Pork - Kidney,3
...,...
Kiwi,1
Knife Plastic - White,1
"Lamb - Pieces, Diced",1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [85]:
a = 0
for product in aggregate.index:
    if product_by_customer_matrix.loc[product][200]==0:
        print(product)
        a += 1
    if a == 5:
        break

Soup - Campbells Bean Medley
Pomello
Bay Leaf
Pork - Kidney
Muffin - Carrot Individual Wrap


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [89]:
recommendations = {}
def recommendation(CustomerID):
    rec = []
    top_5_similar_customers=customer_similarity_matrix[CustomerID].sort_values(ascending=False)[1:6]
    records = total_quantity.loc[top_5_similar_customers.index]
    aggregate = records.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
    a = 0
    for product in aggregate.index:
        if product_by_customer_matrix.loc[product][CustomerID]==0:
            print(product)
            a += 1
        if a == 5:
            break
    recommendations[CustomerID] = rec

In [90]:
for client in product_by_customer_matrix.columns:
    recommendation(client)

Chocolate - Dark
Wine - Charddonnay Errazuriz
Bay Leaf
Oranges - Navel, 72
Wine - Ej Gallo Sierra Valley
Soup - Campbells Bean Medley
Pomello
Bay Leaf
Pork - Kidney
Muffin - Carrot Individual Wrap
Cheese - Parmesan Grated
Pork - Kidney
Crush - Cream Soda
Soup - Campbells Tomato Ravioli
Pepper - White, Ground
Butter - Unsalted
Pepper - White, Ground
Cup - Translucent 7 Oz Clear
Sage - Ground
Pork - Bacon, Double Smoked
Wine - Blue Nun Qualitatswein
Beef - Top Sirloin
Butter - Unsalted
Table Cloth 81x81 White
Pepper - Black, Whole
Cheese - Mozzarella
Ice Cream Bar - Hageen Daz To
Knife Plastic - White
Peas - Pigeon, Dry
Beets - Candy Cane, Organic
Cheese - Parmesan Grated
Pepper - Black, Whole
Sugar - Fine
Olive - Spread Tapenade
Quiche Assorted
Butter - Unsalted
Yoghurt Tubes
Cheese - Boursin, Garlic / Herbs
Pork - Kidney
Lamb - Ground
Ice Cream Bar - Hageen Daz To
Peas - Pigeon, Dry
Squid - Tubes / Tenticles 10/20
Chinese Foods - Chicken
Pepper - Black, Whole
Cod - Black Whole Fillet
B

Beef Wellington
Bar Mix - Pina Colada, 355 Ml
Baking Powder
Pasta - Penne, Rigate, Dry
Wine - Magnotta, Merlot Sr Vqa
Beef - Short Loin
Cheese - Wine
Soup - Campbells Bean Medley
Wine - Chardonnay South
Soup V8 Roasted Red Pepper
Table Cloth 62x114 White
Chips Potato All Dressed - 43g
Nantuket Peach Orange
Cheese Cloth No 100
Curry Paste - Madras
Veal - Sweetbread
Lime Cordial - Roses
Pate - Cognac
Lamb - Pieces, Diced
Sword Pick Asst
Bread - Italian Roll With Herbs
Scallop - St. Jaques
Sausage - Breakfast
Vanilla Beans
Carbonated Water - Blackcherry
Puree - Passion Fruit
Soup - Campbells Bean Medley
Mushroom - Porcini, Dry
Wine - Chardonnay South
Tea - Jasmin Green
Zucchini - Yellow
Olive - Spread Tapenade
Wine - Ej Gallo Sierra Valley
Vaccum Bag 10x13
Longos - Grilled Chicken With
Sardines
Chicken - Leg, Boneless
Thermometer Digital
Curry Paste - Madras
Sole - Dover, Whole, Fresh
Sherry - Dry
Bacardi Breezer - Tropical
Olives - Stuffed
Soup - Campbells Tomato Ravioli
Cheese - Boursin

Tea - Jasmin Green
Coffee - Dark Roast
Ezy Change Mophandle
Spinach - Baby
Cake - Mini Cheesecake
Cocktail Napkin Blue
Mushrooms - Black, Dried
Pasta - Cheese / Spinach Bauletti
Raspberries - Fresh
Berry Brulee
Sauce - Demi Glace
Placemat - Scallop, White
Lettuce - Treviso
Sausage - Breakfast
Durian Fruit
Beans - Wax
Cheese - Victor Et Berthold
Cheese - Cheddarsliced
Cake - Box Window 10x10x2.5
Bread - Italian Corn Meal Poly
Salsify, Organic
Veal - Eye Of Round
Cheese - Wine
Water - Spring Water 500ml
Hersey Shakes
Cheese - Cheddarsliced
Beer - Blue
Rosemary - Dry
Beer - Original Organic Lager
Chocolate - Compound Coating
Salsify, Organic
Curry Paste - Madras
Dried Figs
Water - Aquafina Vitamin
Rambutan
Chicken - Leg, Boneless
Coffee - Hazelnut Cream
Scallops - Live In Shell
Crab - Imitation Flakes
Kiwi
Lettuce - Treviso
Cheese - Mix
Watercress
Sauce - Gravy, Au Jus, Mix
Soup - Campbells Bean Medley
Pail For Lid 1537
Jagermeister
Wine - Valpolicella Masi
Lime Cordial - Roses
Beans - Ki

Onions - Cippolini
Soup Knorr Chili With Beans
Lentils - Red, Dry
Fish - Scallops, Cold Smoked
Black Currants
Grenadine
Coffee - Dark Roast
Hersey Shakes
Lamb - Ground
Beans - Kidney White
Wine - Fume Blanc Fetzer
Salsify, Organic
Mangoes
Bread - Rye
Potatoes - Idaho 100 Count
Muffin Batt - Choc Chk
Squid - Tubes / Tenticles 10/20
Mushroom - Trumpet, Dry
Jagermeister
Chinese Foods - Chicken
Pop Shoppe Cream Soda
Water - Aquafina Vitamin
Ecolab - Solid Fusion
Muffin Batt - Choc Chk
Beef - Striploin Aa
Tea - Jasmin Green
Mussels - Frozen
Mushroom - Porcini, Dry
Black Currants
Cheese - Camembert
Wine - Wyndham Estate Bin 777
Cake - Mini Cheesecake
Pail For Lid 1537
Jagermeister
Wine - Valpolicella Masi
Rosemary - Dry
Wine - Hardys Bankside Shiraz
Chicken - Soup Base
Wonton Wrappers
General Purpose Trigger
Table Cloth 120 Round White
Wiberg Super Cure
Spoon - Soup, Plastic
Nantucket - Pomegranate Pear
Table Cloth 62x114 White
Juice - V8 Splash
Meldea Green Tea Liquor
Beef - Striploin Aa
Mu

Spoon - Soup, Plastic
Beef - Top Sirloin
Soup - Campbells, Cream Of
General Purpose Trigger
Lamb - Ground
Ezy Change Mophandle
Grouper - Fresh
Veal - Osso Bucco
Vaccum Bag 10x13
Tomatoes Tear Drop
Rum - Mount Gay Eclipes
Table Cloth 62x114 White
Liners - Baking Cups
Assorted Desserts
Ice Cream Bar - Hageen Daz To
Beef - Striploin Aa
Veal - Inside
Liners - Baking Cups
Juice - V8 Splash
Bread Crumbs - Panko
Jagermeister
Chocolate - Compound Coating
Garlic
Pail For Lid 1537
Banana Turning
Appetizer - Mushroom Tart
Seedlings - Mix, Organic
Skirt - 29 Foot
Wine - Valpolicella Masi
Cake - Box Window 10x10x2.5
Wine - Magnotta, Merlot Sr Vqa
Cookies - Assorted
Cod - Black Whole Fillet
Cheese - Parmesan Cubes
Puree - Passion Fruit
Barramundi
Soup Knorr Chili With Beans
Pate - Cognac
Grouper - Fresh
Pail For Lid 1537
Bay Leaf
Cup - 6oz, Foam
Lettuce - California Mix
Chips Potato Salt Vinegar 43g
Isomalt
Carbonated Water - Cherry
Beer - Labatt Blue
Rum - Coconut, Malibu
Chocolate - Compound Coati

...

Nantucket - Pomegranate Pear
Tart Shells - Sweet, 4
Pork - Inside
Peas - Frozen
Beer - Rickards Red
Cake - Box Window 10x10x2.5
Bread - French Baquette
Lentils - Red, Dry
Rosemary - Primerba, Paste
Tuna - Salad Premix
Hickory Smoke, Liquid
Cake - Mini Cheesecake
Broom - Corn
Liners - Baking Cups
Bandage - Flexible Neon
Bread - Italian Roll With Herbs
Onion Powder
Mussels - Cultivated
Tea - Earl Grey
Pork - Back, Short Cut, Boneless
Pork - Inside
Wine - Blue Nun Qualitatswein
Veal - Inside
Cake - Cake Sheet Macaroon
Cheese - Mix
Snapple - Iced Tea Peach
Smirnoff Green Apple Twist
Cookies Cereal Nut
Crush - Cream Soda
Beef - Top Sirloin - Aaa
Bacardi Breezer - Tropical
Tea - Jasmin Green
Chinese Foods - Chicken
Fish - Scallops, Cold Smoked
Beef - Prime Rib Aaa
Scallops - Live In Shell
Chips Potato All Dressed - 43g
Mussels - Frozen
Rice - Jasmine Sented
Beer - Labatt Blue
Pastry - Raisin Muffin - Mini
Hickory Smoke, Liquid
Milk Powder
Wine - Valpolicella Masi
Rice - Jasmine Sented
Bread 

Pork - Belly Fresh
Vaccum Bag 10x13
Cake - Box Window 10x10x2.5
Crackers Cheez It
Tomato - Tricolor Cherry
Bananas
Puree - Mocha
Wine - Hardys Bankside Shiraz
Curry Paste - Madras
Baking Powder
Bread - Italian Corn Meal Poly
Smirnoff Green Apple Twist
Coconut - Shredded, Sweet
Pomello
Chef Hat 20cm
Oil - Shortening - All - Purpose
Tart Shells - Sweet, 4
Cookie Chocolate Chip With
Muffin Chocolate Individual Wrap
Scallops 60/80 Iqf
Bacardi Breezer - Tropical
Milk - 1%
Pork - Belly Fresh
Beef - Tenderlion, Center Cut
Smoked Paprika
Chocolate - Dark
Cocoa Butter
Bananas
Crush - Cream Soda
Ketchup - Tomato
Lettuce - Frisee
Wine - Pinot Noir Latour
Soup - Campbells, Beef Barley
Beets - Mini Golden
Wine - Vidal Icewine Magnotta
Rice - Long Grain
Scallops - Live In Shell
Blueberries
Juice - Lime
Thermometer Digital
Bagel - Plain
Apricots Fresh
Island Oasis - Mango Daiquiri
Beef - Prime Rib Aaa
Extract - Lemon
Juice - Cranberry, 341 Ml
Cinnamon Buns Sticky
Chicken - Leg, Boneless
Puree - Mocha

Lettuce - Spring Mix
Coconut - Shredded, Sweet
Salmon Steak - Cohoe 8 Oz
Table Cloth 81x81 White
Ecolab - Mikroklene 4/4 L
Beans - Kidney White
Scallops - 10/20
Banana - Leaves
Broom - Corn
Beets - Candy Cane, Organic
Bouq All Italian - Primerba
Fuji Apples
Crush - Cream Soda
Lettuce - California Mix
Tray - 16in Rnd Blk
Oil - Shortening,liqud, Fry
Fish - Scallops, Cold Smoked
Chocolate - Feathers
Bread - English Muffin
Cassis
Tea - Jasmin Green
Cocoa Butter
Sea Bass - Whole
Spice - Peppercorn Melange
Anchovy Paste - 56 G Tube
Spinach - Baby
Wine - Chardonnay South
Pecan Raisin - Tarts
Bread - Multigrain
Oregano - Dry, Rubbed
Potatoes - Instant, Mashed
Rum - Mount Gay Eclipes
Cheese - Cheddarsliced
Tart Shells - Sweet, 4
Nantucket - Pomegranate Pear
Tomato - Tricolor Cherry
Lentils - Red, Dry
Foam Cup 6 Oz
Brandy - Bar
Muffin Chocolate Individual Wrap
Crackers - Trio
Pastry - Butterscotch Baked
Doilies - 5, Paper
Muffin Mix - Blueberry
Phyllo Dough
Spinach - Baby
Lentils - Red, Dry
Bage

Cheese - Brie, Triple Creme
Longos - Chicken Wings
Soup - Campbells Tomato Ravioli
Spinach - Baby
Placemat - Scallop, White
Papayas
Chocolate - Semi Sweet, Calets
Ocean Spray - Ruby Red
Cheese - Wine
Beef - Chuck, Boneless
Curry Paste - Madras
Longos - Grilled Salmon With Bbq
Soup V8 Roasted Red Pepper
Wine - Alsace Gewurztraminer
Bread - French Baquette
Pork - Back, Short Cut, Boneless
Pastry - Butterscotch Baked
Cake - Mini Cheesecake
Potatoes - Instant, Mashed
Bread - English Muffin
Scallops - Live In Shell
Vol Au Vents
Knife Plastic - White
Tuna - Salad Premix
Coconut - Shredded, Sweet
Beets - Mini Golden
Rosemary - Primerba, Paste
Cassis
Pepper - Paprika, Hungarian
Apricots - Dried
Sobe - Tropical Energy
Wine - White Cab Sauv.on
Towels - Paper / Kraft
Cocktail Napkin Blue
Sole - Dover, Whole, Fresh
Corn Meal
Turkey - Oven Roast Breast
Pecan Raisin - Tarts
Beef Wellington
Thermometer Digital
Pepper - Paprika, Hungarian
Spinach - Baby
Assorted Desserts
Meldea Green Tea Liquor
Shrimp

Lentils - Red, Dry
Oregano - Dry, Rubbed
Foam Cup 6 Oz
Ocean Spray - Kiwi Strawberry
Ocean Spray - Ruby Red
Pastry - Cheese Baked Scones
Tart Shells - Sweet, 4
Sunflower Seed Raw
Bread Foccacia Whole
Broom - Corn
Cheese - Mozzarella
Muffin - Carrot Individual Wrap
Chinese Foods - Chicken
Pail For Lid 1537
Pasta - Detalini, White, Fresh
Skirt - 29 Foot
Water - Green Tea Refresher
Mushroom - Trumpet, Dry
Chicken - Soup Base
Pickerel - Fillets
Peas - Pigeon, Dry
Wine - Vidal Icewine Magnotta
Cake - Box Window 10x10x2.5
Chocolate - Dark
Muffin Chocolate Individual Wrap
Beets - Candy Cane, Organic
Dc - Frozen Momji
Milk - 1%
Lettuce - Treviso
Chips Potato All Dressed - 43g
Bouq All Italian - Primerba
Ecolab - Mikroklene 4/4 L
Cup - 6oz, Foam
Beans - Kidney White
Initation Crab Meat
Lettuce - California Mix
Bar Mix - Pina Colada, 355 Ml
Lime Cordial - Roses
Banana Turning
Beer - Sleemans Cream Ale
Mushrooms - Black, Dried
Olives - Kalamata
Tilapia - Fillets
Beans - Wax
Beef - Top Sirloin
Win

In [92]:
recommendations

{33: [],
 200: [],
 264: [],
 356: [],
 412: [],
 464: [],
 477: [],
 639: [],
 649: [],
 669: [],
 694: [],
 756: [],
 883: [],
 891: [],
 1008: [],
 1034: [],
 1066: [],
 1072: [],
 1336: [],
 1428: [],
 1435: [],
 1534: [],
 1577: [],
 1594: [],
 1754: [],
 1839: [],
 1920: [],
 2187: [],
 2329: [],
 2503: [],
 2556: [],
 2566: [],
 2582: [],
 2617: [],
 2686: [],
 2754: [],
 2776: [],
 2902: [],
 2915: [],
 2939: [],
 3074: [],
 3253: [],
 3267: [],
 3305: [],
 3317: [],
 3472: [],
 3531: [],
 3535: [],
 3544: [],
 3885: [],
 3903: [],
 3909: [],
 4261: [],
 4479: [],
 4595: [],
 4644: [],
 4723: [],
 5104: [],
 5177: [],
 5183: [],
 5224: [],
 5230: [],
 5434: [],
 5678: [],
 5739: [],
 5832: [],
 5921: [],
 5968: [],
 5986: [],
 5997: [],
 6001: [],
 6185: [],
 6317: [],
 6352: [],
 6367: [],
 6447: [],
 6543: [],
 6734: [],
 6817: [],
 6851: [],
 7021: [],
 7134: [],
 7223: [],
 7543: [],
 7606: [],
 7623: [],
 7772: [],
 7827: [],
 7987: [],
 7999: [],
 8316: [],
 8367: [],
 83

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.